In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install splink

In [2]:
from splink import splink_datasets, DuckDBAPI, block_on
import splink.exploratory as exploratory

df = splink_datasets.historical_50k
df = df.rename(columns={"cluster": "cluster_ground_truth"})
df.head(5)


,unique_id,cluster_ground_truth,full_name,first_and_surname,first_name,surname,dob,birth_place,postcode_fake,gender,occupation
0,Q2296770-1,Q2296770,"thomas clifford, 1st baron clifford of chudleigh",thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,male,politician
1,Q2296770-2,Q2296770,thomas of chudleigh,thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,male,politician
2,Q2296770-3,Q2296770,tom 1st baron clifford of chudleigh,tom chudleigh,tom,chudleigh,1630-08-01,devon,tq13 8df,male,politician
3,Q2296770-4,Q2296770,thomas 1st chudleigh,thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8hu,None,politician
4,Q2296770-5,Q2296770,"thomas clifford, 1st baron chudleigh",thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,None,politician


## Exploratory analysis

In [3]:
import splink.exploratory as exploratory

db_api = DuckDBAPI()
exploratory.completeness_chart(df, db_api=db_api)

alt.LayerChart(...)

In [4]:
db_api = DuckDBAPI()
exploratory.profile_columns(df, column_expressions=["first_name", "dob"], db_api=db_api)

alt.VConcatChart(...)

## Blocking analysis

In [5]:
from splink import blocking_analysis as ba
from splink import block_on

blocking_rule_for_analysis = block_on("first_name")

res = ba.count_comparisons_from_blocking_rule(
    table_or_tables=df,
    blocking_rule=blocking_rule_for_analysis,
    link_type="dedupe_only",
    db_api=db_api,
)
res

{'number_of_comparisons_generated_pre_filter_conditions': 32796475,
 'number_of_comparisons_to_be_scored_post_filter_conditions': 16372982,
 'filter_conditions_identified': '',
 'equi_join_conditions_identified': 'l."first_name" = r."first_name"',
 'link_type_join_condition': 'where l."unique_id" < r."unique_id"'}

In [6]:
num_comparisons = res["number_of_comparisons_to_be_scored_post_filter_conditions"]
print(
    f"Number of comparisons generated: {num_comparisons:,}"
)

Number of comparisons generated: 16,372,982


In [7]:
blocking_rule_for_analysis = block_on("substr(first_name, 1, 1)")

res = ba.count_comparisons_from_blocking_rule(
    table_or_tables=df,
    blocking_rule=blocking_rule_for_analysis,
    link_type="dedupe_only",
    db_api=db_api,
)

num_comparisons = res["number_of_comparisons_to_be_scored_post_filter_conditions"]
print(
    f"Number of comparisons generated: {num_comparisons:,}"
)

Number of comparisons generated: 90,801,655


In [8]:
blocking_rules = [
    block_on("first_name", "surname"),
    block_on("first_name", "dob"),
    block_on("dob", "postcode_fake"),
    block_on("occupation", "dob"),
    block_on("birth_place", "dob"),
    block_on("birth_place", "substr(postcode_fake,1,4)"),
    block_on("substr(first_name,1,2)", "surname"),

]

ba.cumulative_comparisons_to_be_scored_from_blocking_rules_chart(
    link_type="dedupe_only",
    blocking_rules=blocking_rules,
    table_or_tables=df,
    db_api=db_api,
)



alt.Chart(...)

In [9]:
blocking_rule_for_analysis = block_on("first_name", "surname")

ba.n_largest_blocks(
    link_type="dedupe_only",
    blocking_rule=blocking_rule_for_analysis,
    table_or_tables=df,
    db_api=db_api,
).as_duckdbpyrelation()

┌─────────┬─────────┬─────────┬─────────┬─────────────┐
│  key_0  │  key_1  │ count_l │ count_r │ block_count │
│ varchar │ varchar │  int64  │  int64  │    int64    │
├─────────┼─────────┼─────────┼─────────┼─────────────┤
│ sir     │ baronet │     542 │     542 │      293764 │
│ 1st     │ bt.     │      57 │      57 │        3249 │
│ 1st     │ baronet │      40 │      40 │        1600 │
│ william │ jones   │      29 │      29 │         841 │
│ john    │ lewis   │      26 │      26 │         676 │
└─────────┴─────────┴─────────┴─────────┴─────────────┘

## Define and estimate a model

```
Data Linking Model
├─-- Comparison: Date of birth
│    ├─-- ComparisonLevel: Exact match on date of birth with term frequency adjustments
│    ├─-- ComparisonLevel: DamerauLevenshtein distance <= 1
│    ├─-- ComparisonLevel: Abs date difference <= 1 month
│    ├─-- ComparisonLevel: Abs date difference <= 1 year
│    ├─-- ComparisonLevel: Abs date difference <= 10 year
│    ├─-- ComparisonLevel: All other comparisons
├─-- Comparison: Birth Place
│    ├─-- ComparisonLevel: Exact match on birth place with term frequency adjustments
│    ├─-- ComparisonLevel: All other
```

In [10]:
import splink.comparison_library as cl
c = cl.DateOfBirthComparison(
            "dob", input_is_string=True
        ).get_comparison("duckdb")

for cl in c.as_dict()['comparison_levels'][1:]:
    print("")
    print(cl["sql_condition"])


"dob_l" = "dob_r"

damerau_levenshtein("dob_l", "dob_r") <= 1

ABS(EPOCH(try_strptime("dob_l", '%Y-%m-%d')) - EPOCH(try_strptime("dob_r", '%Y-%m-%d'))) <= 2629800.0

ABS(EPOCH(try_strptime("dob_l", '%Y-%m-%d')) - EPOCH(try_strptime("dob_r", '%Y-%m-%d'))) <= 31557600.0

ABS(EPOCH(try_strptime("dob_l", '%Y-%m-%d')) - EPOCH(try_strptime("dob_r", '%Y-%m-%d'))) <= 315576000.0

ELSE


In [11]:
import splink.comparison_library as cl

from splink import SettingsCreator

settings = SettingsCreator(
    link_type="dedupe_only",
    blocking_rules_to_generate_predictions=blocking_rules,
    comparisons=[
        cl.ForenameSurnameComparison(
            "first_name",
            "surname",
            forename_surname_concat_col_name="first_and_surname",
        ),
        cl.DateOfBirthComparison(
            "dob", input_is_string=True
        ),
        cl.PostcodeComparison("postcode_fake"),
        cl.ExactMatch("birth_place").configure(term_frequency_adjustments=True),
        cl.ExactMatch("occupation").configure(term_frequency_adjustments=True),
    ],
    retain_intermediate_calculation_columns=True,
    retain_matching_columns=True,
    additional_columns_to_retain=["cluster_ground_truth"]
)

In [12]:
from splink import Linker
linker = Linker(df, settings, db_api=db_api)

### Estimate the probability that two random records match

In [13]:
linker.training.estimate_probability_two_random_records_match(
    [
        "l.first_name = r.first_name and l.surname = r.surname and l.dob = r.dob",
        "substr(l.first_name,1,2) = substr(r.first_name,1,2) and l.surname = r.surname and substr(l.postcode_fake,1,2) = substr(r.postcode_fake,1,2)",
        "l.dob = r.dob and l.postcode_fake = r.postcode_fake",
    ],
    recall=0.6,
)

Probability two random records match is estimated to be  0.000136.
This means that amongst all possible pairwise record comparisons, one in 7,362.31 are expected to match.  With 1,279,041,753 total possible comparisons, we expect a total of around 173,728.33 matching pairs


### Estimate the `u` probabilities (direct estimation)

In [14]:
linker.training.estimate_u_using_random_sampling(max_pairs=1e6)

You are using the default value for `max_pairs`, which may be too small and thus lead to inaccurate estimates for your model's u-parameters. Consider increasing to 1e8 or 1e9, which will result in more accurate estimates, but with a longer run time.
----- Estimating u probabilities using random sampling -----
u probability not trained for first_name_surname - Match on reversed cols: first_name and surname (both directions) (comparison vector value: 5). This usually means the comparison level was never observed in the training data.

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name_surname (some u values are not trained, no m values are trained).
    - dob (no m values are trained).
    - postcode_fake (no m values are trained).
    - birth_place (no m values are trained).
    - occupation (no m values are trained).


### Estimate the `m` probabilities (unsupervised)

In [15]:
training_blocking_rule = block_on("first_name", "surname")
training_session_names = (
    linker.training.estimate_parameters_using_expectation_maximisation(
        training_blocking_rule, estimate_without_term_frequencies=True
    )
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(l."first_name" = r."first_name") AND (l."surname" = r."surname")

Parameter estimates will be made for the following comparison(s):
    - dob
    - postcode_fake
    - birth_place
    - occupation

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name_surname

Iteration 1: Largest change in params was 0.254 in probability_two_random_records_match
Iteration 2: Largest change in params was -0.0884 in the m_probability of postcode_fake, level `Exact match on full postcode`
Iteration 3: Largest change in params was -0.0233 in the m_probability of birth_place, level `Exact match on birth_place`
Iteration 4: Largest change in params was 0.0096 in the m_probability of birth_place, level `All other comparisons`
Iteration 5: Largest change in params was -0.0047 in the m_probability of birth_place, level `Exact match on 

In [16]:
training_blocking_rule = block_on("dob")
training_session_dob = (
    linker.training.estimate_parameters_using_expectation_maximisation(
        training_blocking_rule, estimate_without_term_frequencies=True
    )
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."dob" = r."dob"

Parameter estimates will be made for the following comparison(s):
    - first_name_surname
    - postcode_fake
    - birth_place
    - occupation

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - dob

Iteration 1: Largest change in params was -0.47 in the m_probability of first_name_surname, level `Exact match on first_and_surname`
Iteration 2: Largest change in params was 0.0514 in the m_probability of first_name_surname, level `All other comparisons`
Iteration 3: Largest change in params was 0.0167 in the m_probability of first_name_surname, level `All other comparisons`
Iteration 4: Largest change in params was 0.00504 in the m_probability of first_name_surname, level `All other comparisons`
Iteration 5: Largest change in params was 0.00155 in the m_probability of first_name_surname, level `All o

### Visualise the match weights

In [17]:
linker.visualisations.match_weights_chart()

alt.VConcatChart(...)

### Predict

In [18]:
df_predict = linker.inference.predict(threshold_match_weight=-1)


sql = f"""
select *
from {df_predict.physical_name}
where

(unique_id_l = 'Q7964195-7' and unique_id_r = 'Q7964195-8')
or
(unique_id_l = 'Q2296770-1' and unique_id_r = 'Q2296770-6')

"""

rec = linker.misc.query_sql(sql)
rec

Blocking time: 0.57 seconds
Predict time: 3.52 seconds

 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'first_name_surname':
    u values not fully trained


,match_weight,match_probability,unique_id_l,unique_id_r,surname_l,surname_r,first_name_l,first_name_r,first_and_surname_l,first_and_surname_r,...,occupation_l,occupation_r,gamma_occupation,tf_occupation_l,tf_occupation_r,bf_occupation,bf_tf_adj_occupation,cluster_ground_truth_l,cluster_ground_truth_r,match_key
0,20.094747,0.999999,Q2296770-1,Q2296770-6,chudleigh,chudleigh,thomas,thomas,thomas chudleigh,thomas chudleigh,...,politician,politician,1,0.088932,0.088932,23.542291,0.427871,Q2296770,Q2296770,0
1,1.432043,0.729604,Q7964195-7,Q7964195-8,attenborough,attenborough,walter,walter,walter attenborough,walter attenborough,...,None,None,-1,NaN,NaN,1.000000,1.000000,Q7964195,Q7964195,0


In [19]:
input = rec.to_dict(orient='records')

linker.visualisations.waterfall_chart(input, filter_nulls=False)


alt.LayerChart(...)

### Cluster

In [20]:
clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(df_predict, threshold_match_probability=0.9)
clusters.as_duckdbpyrelation().show(max_width=2000)

Completed iteration 1, num representatives needing updating: 2890
Completed iteration 2, num representatives needing updating: 1150
Completed iteration 3, num representatives needing updating: 509
Completed iteration 4, num representatives needing updating: 237
Completed iteration 5, num representatives needing updating: 109
Completed iteration 6, num representatives needing updating: 51
Completed iteration 7, num representatives needing updating: 21
Completed iteration 8, num representatives needing updating: 7
Completed iteration 9, num representatives needing updating: 2
Completed iteration 10, num representatives needing updating: 0


┌─────────────┬─────────────┬──────────────────────┬──────────────────────────────────────────────────┬───────────────────┬────────────┬───────────┬────────────┬───────────────────┬───────────────┬─────────┬─────────────────────┐
│ cluster_id  │  unique_id  │ cluster_ground_truth │                    full_name                     │ first_and_surname │ first_name │  surname  │    dob     │    birth_place    │ postcode_fake │ gender  │     occupation      │
│   varchar   │   varchar   │       varchar        │                     varchar                      │      varchar      │  varchar   │  varchar  │  varchar   │      varchar      │    varchar    │ varchar │       varchar       │
├─────────────┼─────────────┼──────────────────────┼──────────────────────────────────────────────────┼───────────────────┼────────────┼───────────┼────────────┼───────────────────┼───────────────┼─────────┼─────────────────────┤
│ Q12859389-1 │ Q2296770-1  │ Q2296770             │ thomas clifford, 1st baron 

In [21]:
from IPython.display import IFrame

linker.visualisations.comparison_viewer_dashboard(
    df_predict,
    out_path="comparison_viewer.html",
    overwrite=True,
)


In [22]:
from IPython.display import IFrame

linker.visualisations.cluster_studio_dashboard(
    df_predict,
    clusters,
    "cluster_studio.html",
    sampling_method="by_cluster_size",
    overwrite=True,
)




## Evaluation

In [23]:
linker.evaluation.unlinkables_chart()

alt.LayerChart(...)

In [24]:
linker.evaluation.accuracy_analysis_from_labels_column("cluster_ground_truth", output_type="roc")

Blocking time: 0.80 seconds
Predict time: 1.20 seconds

 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'first_name_surname':
    u values not fully trained


alt.Chart(...)

In [25]:
linker.evaluation.accuracy_analysis_from_labels_column("cluster_ground_truth", output_type="threshold_selection")


Blocking time: 0.71 seconds
Predict time: 0.28 seconds

 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'first_name_surname':
    u values not fully trained


alt.HConcatChart(...)

https://www.robinlinacre.com/splink_example_charts/example_charts/interactive_clusters.html

https://www.robinlinacre.com/splink_example_charts/example_charts/splink3/labelling_tool_dedupe_only.html

https://www.robinlinacre.com/splink_example_charts/example_charts/interactive_clusters.html Kitty muggeridge 137438955949  Rothschild 889
